In [42]:
import tabula
from IPython.display import display
import os
from contextlib import redirect_stderr
import pandas as pd
import numpy as np
import re

# --- Extração do PDF (código original, sem alterações) ---

devnull = open(os.devnull, 'w')
with redirect_stderr(devnull):
    lista_de_tabelas = tabula.read_pdf(
        "Planilha _orcamentaria_analitica.pdf",
        pages="1",
        lattice=True
    )
devnull.close()

df_bruto = pd.DataFrame()
if lista_de_tabelas:
    df_bruto = lista_de_tabelas[0].copy()
    print("--- DataFrame Original Extraído (Ponto de Partida) ---")
    display(df_bruto)
else:
    print("Nenhuma tabela foi encontrada no PDF.")

# --- NOVO: Processamento Inteligente de Cabeçalho e Títulos ---

df_titulos = pd.DataFrame()
df_dados = pd.DataFrame()

if not df_bruto.empty:
    indice_cabecalho = -1
    
    # 1. ENCONTRAR O ÍNDICE DA LINHA DO CABEÇALHO VERDADEIRO (Lógica inalterada)
    print("\n--- Procurando pelo cabeçalho principal nas primeiras 15 linhas... ---")
    for i, row in df_bruto.head(10).iterrows():
        matches = row.astype(str).str.contains('Código|Descri|Valor|Total|Quant|Und', case=False, na=False).sum()
        if matches >= 3:
            indice_cabecalho = i
            print(f"--- Cabeçalho verdadeiro identificado na linha de índice: {indice_cabecalho} ---")
            break

    # 2. SEPARAR O DATAFRAME (Lógica inalterada)
    if indice_cabecalho != -1:
        # Separação inicial: tudo ANTES do cabeçalho vai para uma df temporária de títulos
        df_titulos_temp = df_bruto.iloc[:indice_cabecalho].copy()
        
        novo_cabecalho = df_bruto.iloc[indice_cabecalho]
        df_dados = df_bruto.iloc[indice_cabecalho + 1:].copy()
        df_dados.columns = novo_cabecalho
        
        # 3. LIMPEZA E FILTRAGEM FINAL
        
        # 3a. NOVO: Filtrar os títulos para manter apenas os que têm o formato correto
        if not df_titulos_temp.empty:
            indices_para_manter = []
            # Itera sobre os títulos potenciais para ver quais se encaixam no padrão
            for index, row in df_titulos_temp.iterrows():
                # CONDIÇÃO: A linha deve ter mais de 1 célula com conteúdo.
                # Isso elimina os textos de cabeçalho que só ocupam a primeira coluna.
                if row.notna().sum() > 1:
                    indices_para_manter.append(index)
            
            # Cria o DataFrame de títulos final apenas com as linhas que passaram no filtro
            df_titulos = df_titulos_temp.loc[indices_para_manter].copy()

        # 3b. Limpeza do df_titulos final (lógica anterior, agora aplicada ao df filtrado)
        if not df_titulos.empty:
            df_titulos.dropna(axis=1, how='all', inplace=True)
            df_titulos.columns = [f'Info_{j}' for j in range(len(df_titulos.columns))]
            df_titulos.reset_index(drop=True, inplace=True)
            df_titulos.fillna('', inplace=True)

        # 3c. Limpeza do df_dados (lógica inalterada)
        df_dados.reset_index(drop=True, inplace=True)
        df_dados.fillna('', inplace=True)

    else:
        print("\n--- AVISO: Não foi possível identificar a linha de cabeçalho principal no intervalo de busca. ---")
        df_dados = df_bruto.copy()



    # 4. EXIBIR RESULTADOS
    print("\n--- TÍTULOS EXTRAÍDOS ---")
    if not df_titulos.empty:
        display(df_titulos)
    else:
        print("Nenhuma linha de título foi encontrada.")
        
    print("\n--- DADOS FINAIS (Sem Títulos e sem 'NaN') ---")
    display(df_dados)
    

# --- NOVO: Separar Linhas de Sumário dos Dados Principais ---

df_sumarios_bruto = pd.DataFrame()

if not df_dados.empty:
    indices_sumario = []
    
    # 1. ENCONTRAR OS ÍNDICES DAS LINHAS DE SUMÁRIO
    # Vamos procurar por 'MO sem LS' em qualquer coluna da linha
    for index, row in df_dados.iterrows():
        # .any() retorna True se encontrar a string em qualquer célula da linha
        if row.astype(str).str.contains('MO sem LS', na=False).any():
            indices_sumario.append(index)
            # O sumário às vezes cria linhas vazias adjacentes, vamos pegá-las também
            # Pega a linha seguinte se ela estiver quase toda vazia
            if index + 1 < len(df_dados) and df_dados.iloc[index + 1].notna().sum() < 2:
                 indices_sumario.append(index + 1)

    # 2. CRIAR O DATAFRAME DE SUMÁRIOS E LIMPAR O DF DE DADOS
    if indices_sumario:
        print(f"\n--- Linhas de sumário encontradas nos índices: {indices_sumario}. Separando... ---")
        
        # Cria o novo DataFrame com as linhas de sumário brutas
        df_sumarios_bruto = df_dados.loc[indices_sumario].copy()
        
        # Remove essas linhas do DataFrame de dados principal
        df_dados.drop(index=indices_sumario, inplace=True)
        
        # Limpeza adicional: remove linhas que ficaram totalmente vazias no df_dados
        df_dados.dropna(how='all', inplace=True)
        df_dados.reset_index(drop=True, inplace=True)

        # Limpeza inicial do df de sumários
        df_sumarios_bruto.reset_index(drop=True, inplace=True)

    else:
        print("\n--- Nenhuma linha de sumário ('MO sem LS') foi encontrada nos dados. ---")

# 3. EXIBIR OS RESULTADOS DA SEPARAÇÃO
print("\n--- DADOS FINAIS (Apenas Itens de Composição) ---")
display(df_dados)

print("\n--- SUMÁRIOS BRUTOS EXTRAÍDOS (Prontos para Processamento) ---")
if not df_sumarios_bruto.empty:
    display(df_sumarios_bruto)
else:
    print("Nenhum sumário foi extraído.")


# --- FINAL: Processar e Formatar os Sumários Brutos ---

df_sumarios_processado = pd.DataFrame()

if not df_sumarios_bruto.empty:
    lista_sumarios_processados = []
    
    # Define a estrutura de coordenadas (linha, coluna) para cada item do sumário
    # Coluna 0, 2, 4 para os labels; Coluna 1, 3, 5 para os valores
    mapa_layout = {
        'MO sem LS': (0, 0),
        'LS': (0, 2),
        'MO com LS': (0, 4),
        'Valor do BDI': (1, 0),
        'Valor com BDI': (1, 4),
        'Quant.': (2, 2),
        'Preço Total': (2, 4)
    }

    print("\n--- Processando sumários brutos para o formato final... ---")
    # 1. Itera sobre cada linha do DataFrame de sumários brutos
    for index, row in df_sumarios_bruto.iterrows():
        
        # Concatena todas as células da linha em uma única string de texto
        texto_completo = ''.join(row.dropna().astype(str))
        
        # 2. USA REGEX PARA EXTRAIR TODOS OS PARES (LABEL, VALOR)
        # Este padrão encontra um texto (label), seguido por '=>', seguido por um número (valor)
        # O re.findall nos dará uma lista de tuplas, ex: [('MO sem LS', '38,80'), ('LS', '0,00'), ...]
        padrao = r'([A-Za-z\s\.]+(?:do BDI)?)\s*=>\s*([\d,.]+)'
        pares_extraidos = re.findall(padrao, texto_completo)
        
        # 3. MONTA A TABELA 3x6
        # Cria um "molde" de 3 linhas e 6 colunas, preenchido com células vazias
        grid_sumario = [[""] * 6 for _ in range(3)]
        
        # 4. PREENCHE O MOLDE COM OS DADOS EXTRAÍDOS
        for label, valor in pares_extraidos:
            label = label.strip() # Limpa espaços em branco do label
            if label in mapa_layout:
                linha, coluna_label = mapa_layout[label]
                coluna_valor = coluna_label + 1
                
                # Insere o label e o valor nas posições corretas do grid
                grid_sumario[linha][coluna_label] = label
                grid_sumario[linha][coluna_valor] = valor
        
        # Adiciona o grid formatado (como um DataFrame sem cabeçalho) à nossa lista de resultados
        lista_sumarios_processados.append(pd.DataFrame(grid_sumario))

    # 5. CONCATENA TODOS OS SUMÁRIOS PROCESSADOS EM UM ÚNICO DATAFRAME FINAL
    if lista_sumarios_processados:
        df_sumarios_processado = pd.concat(lista_sumarios_processados, ignore_index=True)
        # Garante que não haja cabeçalho na saída final
        df_sumarios_processado.columns = [""] * len(df_sumarios_processado.columns)


# --- EXIBIÇÃO FINAL ---
print("\n--- SUMÁRIOS PROCESSADOS E FORMATADOS ---")
if not df_sumarios_processado.empty:
    display(df_sumarios_processado)
else:
    print("Nenhum sumário pôde ser processado.")

--- DataFrame Original Extraído (Ponto de Partida) ---


,"ObraBancosB.D.I.Encargos Sociais\rMANUTENÇÃO PREDIAL DO 3° PAVIMENTO – Edf Sede Administrativa\rSINAPI - 03/2025 - Pernambuco22,23%\rNão Desonerado: embutido nos\rORSE - 02/2025 - Sergipepreços unitário dos insumos de\rmão de obra, de acordo com as\rbases.\rPlanilha Orçamentária Analítica",Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1,NaN,NaN,BANHEIRO FEMININO,NaN,NaN,NaN,NaN,"3.980,89"
1,1.1,Código,Banco,Descrição,Tipo,Und,Quant.,Valor Unit,Total
2,Composição,85005,SINAPI,"ESPELHO CRISTAL, ESPESSURA 4MM, COM PARAFUSOS ...",ESQV -\rESQUADRIAS/FERRAGENS/VIDR,m2,"1,0000000","492,58","492,58"
3,Composição\rAuxiliar,88316,SINAPI,SERVENTE COM ENCARGOS COMPLEMENTARES,OSESDI - SERVIÇOS DIVERSOS,H,"0,4000000","22,95","9,18"
4,Composição\rAuxiliar,88325,SINAPI,VIDRACEIRO COM ENCARGOS COMPLEMENTARES,SEDI - SERVIÇOS DIVERSOS,H,"2,0000000","23,11","46,22"
5,Insumo,00000442,SINAPI,"PARAFUSO FRANCES M16 EM ACO GALVANIZADO, COMPR...",Material,UN,"4,0000000","7,17","28,68"
6,Insumo,00011186,SINAPI,ESPELHO CRISTAL E = 4 MM,Material,M2,"1,0000000","408,50","408,50"
7,"MO sem LS =>38,80LS =>0,00MO com LS =>38,80\rV...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.2,Código,Banco,Descrição,Tipo,Und,Quant.,Valor Unit,Total
9,Composição,100849,SINAPI,ASSENTO SANITÁRIO CONVENCIONAL - FORNECIMENTO ...,INHI - INSTALAÇÕES HIDROS\rSANITÁRIAS,UN,"1,0000000","32,65","32,65"



--- Procurando pelo cabeçalho principal nas primeiras 15 linhas... ---
--- Cabeçalho verdadeiro identificado na linha de índice: 1 ---

--- TÍTULOS EXTRAÍDOS ---


,Info_0,Info_1,Info_2
0,1,BANHEIRO FEMININO,"3.980,89"



--- DADOS FINAIS (Sem Títulos e sem 'NaN') ---


...

1,1.1,Código,Banco,Descrição,Tipo,Und,Quant.,Valor Unit,Total
0,Composição,85005,SINAPI,"ESPELHO CRISTAL, ESPESSURA 4MM, COM PARAFUSOS ...",ESQV -\rESQUADRIAS/FERRAGENS/VIDR,m2,"1,0000000","492,58","492,58"
1,Composição\rAuxiliar,88316,SINAPI,SERVENTE COM ENCARGOS COMPLEMENTARES,OSESDI - SERVIÇOS DIVERSOS,H,"0,4000000","22,95","9,18"
2,Composição\rAuxiliar,88325,SINAPI,VIDRACEIRO COM ENCARGOS COMPLEMENTARES,SEDI - SERVIÇOS DIVERSOS,H,"2,0000000","23,11","46,22"
3,Insumo,00000442,SINAPI,"PARAFUSO FRANCES M16 EM ACO GALVANIZADO, COMPR...",Material,UN,"4,0000000","7,17","28,68"
4,Insumo,00011186,SINAPI,ESPELHO CRISTAL E = 4 MM,Material,M2,"1,0000000","408,50","408,50"
5,1.2,Código,Banco,Descrição,Tipo,Und,Quant.,Valor Unit,Total
6,Composição,100849,SINAPI,ASSENTO SANITÁRIO CONVENCIONAL - FORNECIMENTO ...,INHI - INSTALAÇÕES HIDROS\rSANITÁRIAS,UN,"1,0000000","32,65","32,65"
7,Composição\rAuxiliar,88316,SINAPI,SERVENTE COM ENCARGOS COMPLEMENTARES,SEDI - SERVIÇOS DIVERSOS,H,"0,0484000","22,95","1,11"
8,Composição\rAuxiliar,88267,SINAPI,ENCANADOR OU BOMBEIRO HIDRÁULICO COM ENCARGOS\...,SEDI - SERVIÇOS DIVERSOS,H,"0,1536000","27,62","4,24"
9,Insumo,00000377,SINAPI,"ASSENTO SANITARIO DE PLASTICO, TIPO CONVENCIONAL",Material,UN,"1,0000000","27,30","27,30"



--- SUMÁRIOS BRUTOS EXTRAÍDOS (Prontos para Processamento) ---


1,1.1,Código,Banco,Descrição,Tipo,Und,Quant.,Valor Unit,Total
0,"MO sem LS =>38,80LS =>0,00MO com LS =>38,80\rV...",,,,,,,,
1,"MO sem LS =>4,04LS =>0,00MO com LS =>4,04\rVal...",,,,,,,,



--- Processando sumários brutos para o formato final... ---

--- SUMÁRIOS PROCESSADOS E FORMATADOS ---


,,,,,,
0,MO sem LS,"38,80",LS,"0,00",MO com LS,"38,80"
1,Valor do BDI,"109,50",,,Valor com BDI,"602,08"
2,,,Quant.,"1,08",,
3,MO sem LS,"4,04",LS,"0,00",MO com LS,"4,04"
4,Valor do BDI,"7,25",,,Valor com BDI,"39,90"
5,,,Quant.,"2,00",,
